In [5]:
import numpy as np
import pygalmesh
import SimpleITK as sitk
from nanomesh.utils import show_slice, show_volume, generate_mesh_from_binary_image
from nanomesh.periodic_utils import insert_periodic_info

class Square(pygalmesh.DomainBase):
    def __init__(self):
        super().__init__()
        
        self.npts = 51
        self.size = 1
        self.x = np.linspace(0,self.size,self.npts)
        self.data = np.zeros((self.npts,self.npts,self.npts))
        
        
        idx1 = (self.x<0.25*self.size) 
        idx2 = (self.x>0.75*self.size)
        all_idx = list(range(self.npts))
        self.data[np.ix_(idx1,idx1,all_idx)] = 1.
        self.data[np.ix_(idx2,idx2,all_idx)] = 1.
        self.data[np.ix_(idx1,idx2,all_idx)] = 1.
        self.data[np.ix_(idx2,idx1,all_idx)] = 1.
        
                    

    def eval(self, x):
        ix = np.argmin(np.abs(x[0]-self.x))
        iy = np.argmin(np.abs(x[1]-self.x))
        iz = np.argmin(np.abs(x[2]-self.x))
        if self.data[ix,iy,iz] == 1:
            return -1.
        else:
            return 0.


In [6]:
square = Square()
im = sitk.GetImageFromArray(square.data.astype('uint8'))
show_slice(im, dim='z',scale=2)

interactive(children=(IntSlider(value=25, description='z', max=50), Output()), _dom_classes=('widget-interact'…

In [7]:
square.x[square.x < 0.25]

array([0.  , 0.02, 0.04, 0.06, 0.08, 0.1 , 0.12, 0.14, 0.16, 0.18, 0.2 ,
       0.22, 0.24])

In [15]:
mesh = pygalmesh.generate_periodic_mesh(
    Square(),
    [0, 0, 0, 1, 1, 1],
    max_cell_circumradius=0.05,
    min_facet_angle=30,
    max_radius_surface_delaunay_ball=0.05,
    max_facet_distance=0.025,
    max_circumradius_edge_ratio=2.0,
    number_of_copies_in_output=4,
    make_periodic=True,
    # extra_feature_edges=[[[0.24,0.24,0.0],[0.24,0.24,1.]]],
    extra_feature_edges=[[[0.01,0.01,1.],[0.10,0.24,1.],[0.24,0.24,1.],[0.24,0.01,1.]]],
    max_edge_size_at_feature_edges=0.05,
    # odt=True,
    # lloyd=True,
    verbose=False,
)

import pyvista as pv
pv.plot_itk(mesh)

Viewer(geometries=[{'vtkClass': 'vtkPolyData', 'points': {'vtkClass': 'vtkPoints', 'name': '_points', 'numberO…

Viewer(geometries=[{'vtkClass': 'vtkPolyData', 'points': {'vtkClass': 'vtkPoints', 'name': '_points', 'numberO…

In [9]:
mesh

<meshio mesh object>
  Number of points: 35478
  Number of cells:
    triangle: 10544
    tetra: 34952
  Point data: medit:ref
  Cell data: medit:ref

In [10]:
import meshio
meshio.write('cross_periodic.msh', mesh, file_format='gmsh22', binary=False)